In [ ]:
%load_ext autoreload
%autoreload 2

from datasets import load_dataset, load_metric
from data_sets.data_utils import load_hsd_dataset,  get_suite
from utils.results import load_results, get_dataset_scores, get_significance, get_suite_preds, load_hits, metric_max_over_ground_truths, exact_match_score
import json
import pandas as pd
from pathlib import Path
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import hmean
import pickle
import collections
import config

In [ ]:
model_order = ["small", "base", "large", "xl", "xxl", "chatGPT"]

score_order = ["baseline", "Task", "Task+Rules"]

add_order = ["", "+Ex", "(chatGPT)+Ex", "+Rat", "+Ex+Rat"]

method_order = [score + add for score in score_order for add in add_order]

order = {x: i for i, x in enumerate(model_order + method_order)}

In [ ]:
order

In [ ]:
def plot_dataset_results(df, suite=False, values="accuracy"):
    ymin = .9 * df[values].min()
    ymax = 1.1 * df[values].max()
    ylim = (ymin, ymax)
    if suite:
        df = df[['seen' in s or "baseline" in s for s in df.index]].copy()
    
    df['model'] = [x.split("_")[0] for x in df.index]
    df['method'] = [x.split("_")[1] for x in df.index]
    df['method'] = ["Task" if "baseline" in x else f"Task+Rules" for x in df.method]
    df['method'] = [y+"(chatGPT)"  if "from_chatGPT" in x else y for x,y in zip(df.index, df.method)]
    df['method'] =  [y+"+Ex"  if "example" in x else y for x,y in zip(df.index, df.method)]
    df['method'] =  [y+"+Rat"  if "rules" in x else y for x,y in zip(df.index, df.method)]
    df["model"] = df.model.str.split("-").str[-1]
    n_method = len(np.unique(df.method))
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,5))
    return_df = df.copy()
    df = df.pivot(index=["model"], columns="method", values=values)
    df = df.sort_values(by="model", key=lambda x: x.map(order))
    df = df[sorted(df.columns, key= lambda item: order[item])]
    df.plot.bar(ax = ax, ylim=ylim)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, +1.05),
          fancybox=True, shadow=True, ncol=n_method)
    return fig, return_df

In [ ]:
def process_suite_df(df):
    df['model'] = [x.split("_")[0] for x in df.index]
    df['method'] = [x.split("_")[1] for x in df.index]
    df["score"] = df["method"]
    df['method'] = ["Task" if "baseline" in x else f"Task+Rules" for x in df.method]
    df['method'] = [y+"(chatGPT)"  if "from_chatGPT" in x else y for x,y in zip(df.index, df.method)]
    df['method'] =  [y+"+Ex"  if "example" in x else y for x,y in zip(df.index, df.method)]
    df['method'] =  [y+"+Rat"  if "rules" in x else y for x,y in zip(df.index, df.method)]
    df["model"] = df.model.str.split("-").str[-1]
    df = df.set_index(["model", "method"]).pivot(columns="score", values="avg")
    df = df.sort_index(key=lambda x: x.map(order))
    return df

In [ ]:
def compute_gs(dataset_df, suite_df, dataset_metric, scale=100):
    dataset_df = dataset_df.sort_values(by=["model", "method"], key=lambda x: x.map(order))
    dataset_df = dataset_df.set_index(["model", "method"])
    suite_df = process_suite_df(suite_df)
    merge_df = pd.concat([dataset_df, suite_df], axis=1)
    df = pd.DataFrame()
    df.index = merge_df.index
    
    
    df["baseline"] = hmean([merge_df[dataset_metric]*scale, merge_df["baseline"]])
    df["seen"] = hmean([merge_df[dataset_metric]*scale, merge_df["seen"]])
    df["funcOut"] = hmean([merge_df[dataset_metric]*scale, merge_df["funcOut"]])
    df["classOut"] = hmean([merge_df[dataset_metric]*scale, merge_df["classOut"]])
    for method in ["Task", "Task+Ex"]:
        for score in ["seen", "funcOut", "classOut"]:
            df.loc[(slice(None), method), score] = df.loc[(slice(None), method), "baseline"]
    del df["baseline"]
    return df.sort_values(by=["model", "method"], key=lambda x: x.map(order))

In [ ]:
def compare_func_scores(df):
    ret = {}
    comps = [("Task+Rules", "Task"),
             ("Task+Rules+Ex", "Task+Ex"),
             ("Task+Rules(chatGPT)+Ex", "Task+Ex"),
             ("Task+Rules+Rat", "Task"),
             ("Task+Rules+Ex+Rat", "Task+Ex")]
    scores = ["seen", "funcOut", "classOut"]
    for m, b in comps:
        for s in scores:
            ret[f"improv_{m}_{s}"] = pd.DataFrame((df.select_dtypes(include=np.number)[(df.method==m) & (df.score== s)] - df.select_dtypes(include=np.number)[(df.method==b)].values > 0).sum(1)).values.reshape(-1)
            ret[f"same_{m}_{s}"] = pd.DataFrame((df.select_dtypes(include=np.number)[(df.method==m) & (df.score== s)] - df.select_dtypes(include=np.number)[(df.method==b)].values == 0).sum(1)).values.reshape(-1)
            ret[f"worse_{m}_{s}"] = pd.DataFrame((df.select_dtypes(include=np.number)[(df.method==m) & (df.score== s)] - df.select_dtypes(include=np.number)[(df.method==b)].values < 0).sum(1)).values.reshape(-1)
    ret_df = pd.DataFrame.from_dict(ret)
    ret_df.index = ["small", "base", "large", "xl", "xxl", "chatGPT"]
    return ret_df

In [ ]:
all_results = {}

In [ ]:
suite_results = {}

## Sentiment Analysis

### Dataset

In [ ]:
result_path = Path(f"./results/sa/sst2/")

In [ ]:
results = load_results(result_path)

In [ ]:
results = {k: v for k, v in results.items() if "seen" in k or "baseline" in k}

In [ ]:
dataset_test = load_dataset("glue", "sst2")["validation"]

In [ ]:
metric = load_metric("glue","sst2")

In [ ]:
dataset_scores, preds = get_dataset_scores("sa", results, dataset_test["label"], metric)

In [ ]:
sort = sorted(dataset_scores.items(), key=lambda item: item[1]["accuracy"], reverse=True)

In [ ]:
sort

In [ ]:
df = pd.DataFrame.from_dict(dataset_scores, orient="index")

In [ ]:
fig, sst_df = plot_dataset_results(df)

In [ ]:
fig.savefig(f"../specification-instruction-paper/media/sa_dataset.pdf", bbox_inches = "tight")

### Suite

In [ ]:
result_path = Path("./results/sa/suite/")

In [ ]:
results = load_results(result_path, file_type="csv")

In [ ]:
results = {k: v for k, v in results.items() if "seen" in k or "baseline" in k or "funcOut" in k or "classOut" in k}

In [ ]:
df =pd.concat([x for x in results.values()])

In [ ]:
df["avg"].sort_values()

In [ ]:
suite_results["sa"] = process_suite_df(df)

In [ ]:
df[(df.model == "chatGPT") & ((df.method == "Task+Rules+Ex") | (df.method=="Task+Rules+Ex+Rat")) & (df.score=="seen")].T

In [ ]:
df = df.sort_values(by=["model", "method"], key=lambda x: x.map(order))
comps = compare_func_scores(df)

In [ ]:
comps.T

In [ ]:
df.select_dtypes(include=np.number)[(df.method=="Task+Rules") & (df.score== "seen")] - df.select_dtypes(include=np.number)[(df.method=="Task")].values

In [ ]:
fig, sa_df = plot_dataset_results(df, suite=True, values="avg")

In [ ]:
fig.savefig(f"../specification-instruction-paper/media/sa_suite.pdf", bbox_inches = "tight")

In [ ]:
all_results["sa"] = compute_gs(sst_df, df, "accuracy")

In [ ]:
all_results["sa"]

In [ ]:
# pvalues = get_pvalues_suite("sa", models=df.index.str.split("_").str[0].unique())

In [ ]:
# df = pd.DataFrame.from_dict(pvalues, orient="index")
# df < .05

## Hate Speech detection

### Datasets

In [ ]:
davidson_path = Path(f"./results/hsd/davidson2017/")
founta_path = Path(f"./results/hsd/founta2018/")

In [ ]:
davidson_results = load_results(davidson_path)
founta_results = load_results(founta_path)

In [ ]:
davidson_test = load_hsd_dataset("davidson2017")["test"]
founta_test = load_hsd_dataset("founta2018")["test"]

In [ ]:
metric = load_metric("glue","qqp")

In [ ]:
dataset_scores, preds = get_dataset_scores("hsd", davidson_results, davidson_test["label"], metric)

In [ ]:
dataset_scores = {k: v for k, v in dataset_scores.items() if "seen" in k or "baseline" in k}

In [ ]:
sort = sorted(dataset_scores.items(), key=lambda item: item[1]["f1"], reverse=True)

In [ ]:
sort

In [ ]:
df = pd.DataFrame.from_dict(dataset_scores, orient="index")

In [ ]:
fig, davidson_df = plot_dataset_results(df, values="f1")

In [ ]:
fig.savefig(f"../specification-instruction-paper/media/hsd_davidson.pdf", bbox_inches = "tight")

In [ ]:
# pvalues = get_pvalues_dataset(preds, "hsd", "davidson2017", davidson_test)

In [ ]:
# df = pd.DataFrame.from_dict(pvalues, orient="index")
# df < .05

In [ ]:
dataset_scores, preds = get_dataset_scores("hsd", founta_results, founta_test["label"], metric)

In [ ]:
dataset_scores = {k: v for k, v in dataset_scores.items() if "seen" in k or "baseline" in k}

In [ ]:
sort = sorted(dataset_scores.items(), key=lambda item: item[1]["f1"], reverse=True)

In [ ]:
sort

In [ ]:
df = pd.DataFrame.from_dict(dataset_scores, orient="index")

In [ ]:
fig, founta_df = plot_dataset_results(df, values="f1")

In [ ]:
fig.savefig(f"../specification-instruction-paper/media/hsd_founta.pdf", bbox_inches = "tight")

In [ ]:
# pvalues = get_pvalues_dataset(preds, "hsd", "founta2018", founta_test)

In [ ]:
# df = pd.DataFrame.from_dict(pvalues, orient="index")
# df < .05

### Suite

In [ ]:
result_path = Path("./results/hsd/suite/")

In [ ]:
results = load_results(result_path, hatecheck=True)

In [ ]:
results = {k: v for k, v in results.items() if "seen" in k or "baseline" in k or "funcOut" in k or "classOut" in k}

In [ ]:
df =pd.concat([x for x in results.values()])

In [ ]:
df *= 100

In [ ]:
df[df.index.str.contains("flan-t5-xxl")]["avg"].sort_values()

In [ ]:
suite_results["hsd"] = process_suite_df(df)

In [ ]:
fig, hsd_df = plot_dataset_results(df, suite=True, values="avg")

In [ ]:
fig.savefig(f"../specification-instruction-paper/media/hsd_suite.pdf", bbox_inches = "tight")

In [ ]:
all_results["hsd_d"] = compute_gs(davidson_df, df, dataset_metric="f1"); all_results["hsd_d"]

In [ ]:
all_results["hsd_f"] = compute_gs(founta_df, df, dataset_metric="f1"); all_results["hsd_f"]

In [ ]:
# pvalues = get_pvalues_suite("hsd", models=df.index.str.split("_").str[0].unique())

In [ ]:
# df = pd.DataFrame.from_dict(pvalues, orient="index")
# df < .05

## Reading comprehension

### Dataset

In [ ]:
result_path = Path(f"./results/rc/squad/")

In [ ]:
results = load_results(result_path)

In [ ]:
results = {k: v for k, v in results.items() if "seen" in k or "baseline" in k}

In [ ]:
dataset_test = load_dataset("squad")["validation"]

In [ ]:
metric = load_metric("squad")

In [ ]:
dataset_scores, preds = get_dataset_scores("rc", results, dataset_test["answers"], metric)

In [ ]:
sort = sorted(dataset_scores.items(), key=lambda item: item[1]["exact_match"], reverse=True); sort

In [ ]:
# from utils.results import normalize_answer

# hits = {}
# for model, preds in results.items():
#     for answers, pred in zip(dataset_test["answers"], preds):
#         hit = 0
#         for answer in answers["text"]:
#             if normalize_answer(answer) in normalize_answer(pred):
#                 hit = 1
#                 break
#         hits.setdefault(model, []).append(hit)
    

In [ ]:
# for model, hit in hits.items():
#     hits[model] = {}
#     hits[model]["recall"] = np.mean(hit)

In [ ]:
# sorted(hits.items(), key=lambda item: item[1]["recall"], reverse=True)

In [ ]:
df = pd.DataFrame.from_dict(dataset_scores, orient="index")

In [ ]:
fig, squad_df = plot_dataset_results(df, values="exact_match")

In [ ]:
fig.savefig(f"../specification-instruction-paper/media/rc_dataset.pdf", bbox_inches = "tight")

In [ ]:
# pvalues = get_pvalues_dataset(preds, "rc", "squad", dataset_test)

In [ ]:
# df = pd.DataFrame.from_dict(pvalues, orient="index")
# df < .05

### Suite

In [ ]:
result_path = Path("./results/rc/suite/")

In [ ]:
results = load_results(result_path, file_type="csv")

In [ ]:
results = {k: v for k, v in results.items() if "seen" in k or "baseline" in k or "funcOut" in k or "classOut" in k}

In [ ]:
# from utils.results import normalize_answer
# from collections import Counter

# def get_recall(prediction, ground_truth):
#     prediction_tokens = normalize_answer(prediction).split()
#     ground_truth_tokens = normalize_answer(ground_truth).split()
#     common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
#     num_same = sum(common.values())
#     if num_same == 0:
#         return 0
#     precision = 1.0 * num_same / len(prediction_tokens)
#     recall = 1.0 * num_same / len(ground_truth_tokens)
#     f1 = (2 * precision * recall) / (precision + recall)
#     return recall

In [ ]:
# from tqdm.auto import tqdm
# from data_sets.data_utils import get_suite
# import config
# from utils.results import f1_score

# suite_test = get_suite(config.rc_path)["test"]
# f1s = {}
# results_raw = load_results(result_path, file_type="json")
# results_raw = {k: v for k, v in results_raw.items() if "seen" in k or "baseline_zero" in k or "funcOut" in k or "classOut" in k}
# results_raw = get_suite_preds(results_raw, "rc")

In [ ]:
# for model, preds in tqdm(results_raw.items()):
#     for example, pred in zip(suite_test, preds):
#         if example["answers"]["answer_start"][0] == -1:
#             continue
#         func = example["functionality"]
#         answer = example["answers"]["text"][0]
#         f1 = get_recall(pred, answer)
#         f1s.setdefault(model, {}).setdefault(func, []).append(f1)

In [ ]:
# for model, scores in f1s.items():
#     for func, score in scores.items():
#         f1s[model][func] = np.mean(score) * 100

In [ ]:
# for model, scores in f1s.items():
#     for func, score in scores.items():
#         results[model][func] = score 

In [ ]:
df =pd.concat([x for x in results.values()])

In [ ]:
# del df["avg"]
# df["avg"] = df.mean(axis=1)

In [ ]:
df["avg"].sort_values()

In [ ]:
suite_results["rc"] = process_suite_df(df)

In [ ]:
fig, rc_df = plot_dataset_results(df, suite=True, values="avg")

In [ ]:
fig.savefig(f"../specification-instruction-paper/media/rc_suite.pdf", bbox_inches = "tight")

In [ ]:
all_results["rc"] = compute_gs(squad_df, df, dataset_metric="exact_match", scale=1.0); all_results["rc"]

In [ ]:
# pvalues = get_pvalues_suite("rc", models=df.index.str.split("_").str[0].unique())

In [ ]:
# df = pd.DataFrame.from_dict(pvalues, orient="index")
# df < .05

## Paraphrase identification

### Dataset

In [ ]:
result_path = Path(f"./results/pi/qqp/")

In [ ]:
results = load_results(result_path)

In [ ]:
results = {k: v for k, v in results.items() if "seen" in k or "baseline" in k}

In [ ]:
dataset_test = load_dataset("glue", "qqp")["validation"]

In [ ]:
metric = load_metric("glue", "qqp")

In [ ]:
dataset_scores, preds = get_dataset_scores("pi", results, dataset_test["label"], metric)

In [ ]:
sort = sorted(dataset_scores.items(), key=lambda item: item[1]["accuracy"], reverse=True)

In [ ]:
sort

In [ ]:
df = pd.DataFrame.from_dict(dataset_scores, orient="index")

In [ ]:
fig, qqp_df = plot_dataset_results(df)

In [ ]:
fig.savefig(f"../specification-instruction-paper/media/pi_dataset.pdf", bbox_inches = "tight")

In [ ]:
# pvalues = get_pvalues_dataset(preds, "pi", "qqp", dataset_test)

In [ ]:
# df = pd.DataFrame.from_dict(pvalues, orient="index")
# df < .05

### Suite

In [ ]:
result_path = Path("./results/pi/suite/")

In [ ]:
results = load_results(result_path, file_type="csv")

In [ ]:
results = {k: v for k, v in results.items() if "seen" in k or "baseline" in k or "funcOut" in k or "classOut" in k}

In [ ]:
df =pd.concat([x for x in results.values()])

In [ ]:
suite_results["pi"] = process_suite_df(df)

In [ ]:
fig, pi_df = plot_dataset_results(df, suite=True, values="avg")

In [ ]:
fig.savefig(f"../specification-instruction-paper/media/pi_suite.pdf", bbox_inches = "tight")

In [ ]:
compute_gs(qqp_df, df, dataset_metric="accuracy")

In [ ]:
all_results["pi"] = compute_gs(qqp_df, df, dataset_metric="accuracy"); all_results["pi"]

In [ ]:
# pvalues = get_pvalues_suite("pi", models=df.index.str.split("_").str[0].unique())

In [ ]:
# df = pd.DataFrame.from_dict(pvalues, orient="index")
# df < .05

## Table

In [ ]:
import collections

order_task = {"sa": 0, "pi": 1, "rc": 2, "hsd_d": 3, "hsd_f":4}

In [ ]:
od = collections.OrderedDict(sorted(all_results.items(), key= lambda item: order_task[item[0]]))

In [ ]:
for task, df in od.items():
    df.columns = [f"{col}_{task}" for col in df.columns]
    

In [ ]:
all_gs = pd.concat([df for df in od.values()], names=all_results.keys(), axis=1)

In [ ]:
all_gs = all_gs.sort_values(by=["model", "method"], key=lambda x: x.map(order))

In [ ]:
all_gs

In [ ]:
all_gs["avg"] = all_gs.mean(1)

In [ ]:
all_gs["avg"].sort_values()

In [ ]:
all_gs["avg"]

In [ ]:
all_gs.to_csv("./results/all_gs.csv")

In [ ]:
all_gs = pd.read_csv("./results/all_gs.csv", index_col=[0,1])

In [ ]:
all_gs

In [ ]:
sa_df.to_csv("./results/sa_df.csv")
pi_df.to_csv("./results/pi_df.csv")
rc_df.to_csv("./results/rc_df.csv")
hsd_df.to_csv("./results/hsd_df.csv")
sst_df.to_csv("./results/sst_df.csv")
qqp_df.to_csv("./results/qqp_df.csv")
squad_df.to_csv("./results/squad_df.csv")
davidson_df.to_csv("./results/davidson_df.csv")
founta_df.to_csv("./results/founta_df.csv")

In [ ]:
sa_df = pd.read_csv("./results/sa_df.csv", index_col=0)
pi_df = pd.read_csv("./results/pi_df.csv", index_col=0)
rc_df = pd.read_csv("./results/rc_df.csv", index_col=0)
hsd_df = pd.read_csv("./results/hsd_df.csv", index_col=0)
sst_df = pd.read_csv("./results/sst_df.csv", index_col=0)
qqp_df = pd.read_csv("./results/qqp_df.csv", index_col=0)
squad_df = pd.read_csv("./results/squad_df.csv", index_col=0)
davidson_df = pd.read_csv("./results/davidson_df.csv", index_col=0)
founta_df = pd.read_csv("./results/founta_df.csv", index_col=0)

In [ ]:
with open("./results/all_suite_avgs.pkl", "wb") as file:
    pickle.dump(suite_results, file)

In [ ]:
with open("./results/all_suite_avgs.pkl", "rb") as file:
    suite_results = pickle.load(file)

### Some statistics

In [ ]:
# Rule improvement (no demonstrations)
for v in all_gs["avg"].xs("Task+Rules", level=1) - all_gs["avg"].xs("Task", level=1):
    print(np.round(v, 2))

In [ ]:
# Rule improvemen (with demonstrations)
for v in all_gs["avg"].xs("Task+Rules+Ex", level=1) - all_gs["avg"].xs("Task+Ex", level=1):
    print(np.round(v, 2))

In [ ]:
# impact of demonstrations
for v in all_gs["avg"].xs("Task+Ex", level=1) - all_gs["avg"].xs("Task", level=1):
    print(np.round(v, 2))

In [ ]:
# impact of demonstrations
for v in all_gs["avg"].xs("Task+Rules+Ex", level=1) - all_gs["avg"].xs("Task+Rules", level=1):
    print(np.round(v, 2))

In [ ]:
# impact of demonstrations
for v in all_gs["avg"].xs("Task+Rules+Ex+Rat", level=1) - all_gs["avg"].xs("Task+Rules+Rat", level=1):
    print(np.round(v, 2))

In [ ]:
# impact of demonstrations
(all_gs.xs("Task+Ex", level=1) - all_gs.xs("Task", level=1)).mean()

In [ ]:
(all_gs.xs("Task+Rules+Ex", level=1) - all_gs.xs("Task+Rules", level=1)).mean()

In [ ]:
(all_gs.xs("Task+Rules+Ex+Rat", level=1) - all_gs.xs("Task+Rules+Rat", level=1)).mean()

In [ ]:
# impact of rationales
for v in all_gs["avg"].xs("Task+Rules+Rat", level=1) - all_gs["avg"].xs("Task+Rules", level=1):
    print(np.round(v, 2))

In [ ]:
# impact of Rationales
for v in all_gs["avg"].xs("Task+Rules+Ex+Rat", level=1) - all_gs["avg"].xs("Task+Rules+Ex", level=1):
    print(np.round(v, 2))

In [ ]:
# impact of rationales
(all_gs.xs("Task+Rules+Rat", level=1) - all_gs.xs("Task+Rules", level=1)).mean()

In [ ]:
(all_gs.xs("Task+Rules+Ex+Rat", level=1) - all_gs.xs("Task+Rules+Ex", level=1)).mean()

In [ ]:
sa_df.select_dtypes(include=np.number).loc["flan-t5-large_seen"] - sa_df.select_dtypes(include=np.number).loc["flan-t5-large_baseline_zero"]

In [ ]:
dataset_results = pd.concat([sst_df["model"], sst_df["method"], sst_df["accuracy"]*100, qqp_df["accuracy"]*100, squad_df["exact_match"], davidson_df["f1"]*100, founta_df["f1"]*100], axis=1)

In [ ]:
dataset_results["avg"] = dataset_results.mean(1, numeric_only=True)

In [ ]:
suites_df = pd.concat(suite_results.values(), axis=1)

In [ ]:
suites_df["avg"] = suites_df.mean(1, numeric_only=True)

In [ ]:
dataset_results.select_dtypes(include=np.number)[dataset_results["method"] == "Task+Rules+Ex"] - dataset_results.select_dtypes(include=np.number)[dataset_results["method"] == "Task+Ex"].values

In [ ]:
(dataset_results.select_dtypes(include=np.number)[dataset_results["method"] == "Task+Ex"] - dataset_results.select_dtypes(include=np.number)[dataset_results["method"] == "Task"].values).mean()

In [ ]:
(dataset_results.select_dtypes(include=np.number)[dataset_results["method"] == "Task+Rules+Ex"] - dataset_results.select_dtypes(include=np.number)[dataset_results["method"] == "Task+Rules"].values).mean()

In [ ]:
(dataset_results.select_dtypes(include=np.number)[dataset_results["method"] == "Task+Rules+Ex+Rat"] - dataset_results.select_dtypes(include=np.number)[dataset_results["method"] == "Task+Rules+Rat"].values).mean()

In [ ]:
(dataset_results.select_dtypes(include=np.number)[dataset_results["method"] == "Task+Rules+Rat"] - dataset_results.select_dtypes(include=np.number)[dataset_results["method"] == "Task+Rules"].values).mean()

In [ ]:
(dataset_results.select_dtypes(include=np.number)[dataset_results["method"] == "Task+Rules+Ex+Rat"] - dataset_results.select_dtypes(include=np.number)[dataset_results["method"] == "Task+Rules+Ex"].values).mean()

In [ ]:
for suite in suite_results.values():
    print(suite.select_dtypes(include=np.number).xs("Task+Rules+Ex", level=1)["seen"] - suite.select_dtypes(include=np.number).xs("Task+Ex", level=1)["baseline"])

In [ ]:
suites_df.select_dtypes(include=np.number).xs("Task+Rules+Ex", level=1)["seen"].mean(1) - suites_df.select_dtypes(include=np.number).xs("Task+Ex", level=1)["baseline"].mean(1)

In [ ]:
for suite in suite_results.values():
    print((suite.select_dtypes(include=np.number).xs("Task+Ex", level=1).mean(1) - suite.select_dtypes(include=np.number).xs("Task", level=1).mean(1)).mean())

In [ ]:
for suite in suite_results.values():
    print((suite.select_dtypes(include=np.number).xs("Task+Rules+Ex", level=1).mean(1) - suite.select_dtypes(include=np.number).xs("Task+Rules", level=1).mean(1)).mean())

In [ ]:
for suite in suite_results.values():
    print((suite.select_dtypes(include=np.number).xs("Task+Rules+Ex+Rat", level=1).mean(1) - suite.select_dtypes(include=np.number).xs("Task+Rules+Rat", level=1).mean(1)).mean())

In [ ]:
for suite in suite_results.values():
    print((suite.select_dtypes(include=np.number).xs("Task+Rules+Rat", level=1).mean(1) - suite.select_dtypes(include=np.number).xs("Task+Rules", level=1).mean(1)).mean())

In [ ]:
for suite in suite_results.values():
    print((suite.select_dtypes(include=np.number).xs("Task+Rules+Ex+Rat", level=1).mean(1) - suite.select_dtypes(include=np.number).xs("Task+Rules+Ex", level=1).mean(1)).mean())

In [ ]:
all_gs.xs("Task",level=1)

In [ ]:
ruleMethods = ["Task+Rules", "Task+Rules+Ex", "Task+Rules(cGPT)+Ex", "Task+Rules+Rat", "Task+Rules+Ex+Rat"]

In [ ]:
ruleMethodsDf = all_gs.loc[pd.IndexSlice[:, ruleMethods], :]

In [ ]:
# impact on unseen cases
for v in (ruleMethodsDf.iloc[:,range(0, len(all_gs.columns) - 1, 3)] - ruleMethodsDf.iloc[:,range(1, len(all_gs.columns) - 1, 3)].values).mean().values:
    print(np.round(v, 2))

In [ ]:
# impact on unseen cases
(ruleMethodsDf.iloc[:,range(1, len(all_gs.columns) - 1, 3)] - ruleMethodsDf.iloc[:,range(0, len(all_gs.columns) - 1, 3)].values).mean().mean()

In [ ]:
# impact on unseen cases
for v in (ruleMethodsDf.iloc[:,range(0, len(all_gs.columns) - 1, 3)] - ruleMethodsDf.iloc[:,range(2, len(all_gs.columns) - 1, 3)].values).mean().values:
    print(np.round(v, 2))

In [ ]:
# impact on unseen cases
(ruleMethodsDf.iloc[:,range(2, len(all_gs.columns) - 1, 3)] - ruleMethodsDf.iloc[:,range(0, len(all_gs.columns) - 1, 3)].values).mean().mean()

In [ ]:
ruleMethodsDf.iloc[:,range(0, len(all_gs.columns) - 1, 3)].mean()

In [ ]:
ruleMethodsDf.iloc[:,range(1, len(all_gs.columns) - 1, 3)].mean()

In [ ]:
ruleMethodsDf.iloc[:,range(2, len(all_gs.columns) - 1, 3)].mean()

In [ ]:
for v in (ruleMethodsDf.iloc[:,range(0, len(all_gs.columns) - 1, 3)] - ruleMethodsDf.iloc[:,range(1, len(all_gs.columns) - 1, 3)].values >=0).mean().values:
    print(np.round(v*100, 2))

In [ ]:
for v in (all_gs.iloc[:,range(0, len(all_gs.columns) - 1, 3)] - all_gs.iloc[:,range(2, len(all_gs.columns) - 1, 3)].values >=0).mean():
    print(np.round(v*100, 2))

In [ ]:
# Rule improvemen (with demonstrations)
for v in all_gs["avg"].xs("Task+Rules+Ex", level=1) - all_gs["avg"].xs("Task+Ex", level=1):
    print(np.round(v, 2))

In [ ]:
# ChatGPT rules vs human rules

(all_gs.xs("Task+Rules(chatGPT)+Ex", level=1) - all_gs.xs("Task+Rules+Ex", level=1)).mean(1)

In [ ]:
(all_gs.xs("Task+Rules(chatGPT)+Ex", level=1) - all_gs.xs("Task+Ex", level=1)).mean(1)

In [ ]:
all_gs.xs("chatGPT", level=0).iloc[:,range(0, len(all_gs.columns) - 1, 3)] 

In [ ]:
all_gs.xs("chatGPT", level=0).iloc[:,range(0, len(all_gs.columns) - 1, 3)].mean(1) 

In [ ]:
all_gs.xs("chatGPT", level=0).iloc[:,range(1, len(all_gs.columns) - 1, 3)].mean(1)

In [ ]:
all_gs.xs("chatGPT", level=0).iloc[:,range(2, len(all_gs.columns) - 1, 3)].mean(1)

### Table

In [ ]:
with open(f"./results/pvalues.pickle", "rb") as file:
        pvalues = pickle.load(file)

In [ ]:
pvalues_df = pd.DataFrame.from_dict(pvalues, orient="index").reset_index()

In [ ]:
pvalues_df = pvalues_df.rename(columns  ={"level_0": "model", "level_1":"method"})

In [ ]:
pvalues_df = pd.concat([pvalues_df.drop('sa', axis=1), pd.DataFrame(pvalues_df['sa'].tolist()).rename(columns={"seen": "seen_sa",
                                                                                                  "funcOut": "funcOut_sa",
                                                                                                  "classOut": "classOut_sa"})], axis=1)

In [ ]:
pvalues_df = pd.concat([pvalues_df.drop('pi', axis=1), pd.DataFrame(pvalues_df['pi'].tolist()).rename(columns={"seen": "seen_pi",
                                                                                                  "funcOut": "funcOut_pi",
                                                                                                  "classOut": "classOut_pi"})], axis=1)

In [ ]:
pvalues_df = pd.concat([pvalues_df.drop('rc', axis=1), pd.DataFrame(pvalues_df['rc'].tolist()).rename(columns={"seen": "seen_rc",
                                                                                                  "funcOut": "funcOut_rc",
                                                                                                  "classOut": "classOut_rc"})], axis=1)

In [ ]:
pvalues_df = pd.concat([pvalues_df.drop('hsd_f', axis=1), pd.DataFrame(pvalues_df['hsd_f'].tolist()).rename(columns={"seen": "seen_hsd_f",
                                                                                                  "funcOut": "funcOut_hsd_f",
                                                                                                  "classOut": "classOut_hsd_f"})], axis=1)

In [ ]:
pvalues_df = pd.concat([pvalues_df.drop('hsd_d', axis=1), pd.DataFrame(pvalues_df['hsd_d'].tolist()).rename(columns={"seen": "seen_hsd_d",
                                                                                                  "funcOut": "funcOut_hsd_d",
                                                                                                  "classOut": "classOut_hsd_d"})], axis=1)

In [ ]:
pvalues_df = pd.concat([pvalues_df.drop('avg', axis=1), pvalues_df['avg']], axis=1)

In [ ]:
method_map = {
    "": "Task+Rules",
    "example": "Task+Rules+Ex",
    "from_chatGPT_example": "Task+Rules(chatGPT)+Ex",
    "with_rules": "Task+Rules+Rat",
    "example_with_rules": "Task+Rules+Ex+Rat"
}

In [ ]:
pvalues_df['model'] = [x.split("-")[-1] for x in pvalues_df.model] 
pvalues_df['method'] = [method_map[x] for x in pvalues_df.method]

In [ ]:
pvalues_df = pvalues_df.set_index(["model", "method"])

In [ ]:
table = pvalues_df.style

In [ ]:
table = table.format(precision=3)

In [ ]:
print(table.to_latex(multirow_align="t"))

In [ ]:
pvalues_df.loc[("small", "Task"), :] = 16*[1.0]
pvalues_df.loc[("base", "Task"), :] = 16*[1.0]
pvalues_df.loc[("large", "Task"), :] = 16*[1.0]
pvalues_df.loc[("xl", "Task"), :] = 16*[1.0]
pvalues_df.loc[("xxl", "Task"), :] = 16*[1.0]
pvalues_df.loc[("chatGPT", "Task"), :] = 16*[1.0]

In [ ]:
pvalues_df.loc[("small", "Task+Ex"), :] = 16*[1.0]
pvalues_df.loc[("base", "Task+Ex"), :] = 16*[1.0]
pvalues_df.loc[("large", "Task+Ex"), :] = 16*[1.0]
pvalues_df.loc[("xl", "Task+Ex"), :] = 16*[1.0]
pvalues_df.loc[("xxl", "Task+Ex"), :] = 16*[1.0]
pvalues_df.loc[("chatGPT", "Task+Ex"), :] = 16*[1.0]

In [ ]:
pvalues_df = pvalues_df.sort_index(key= lambda x: x.map(order))

In [ ]:
significant = pvalues_df < .05

In [ ]:
l = all_gs.style

In [ ]:
l = l.format(precision=2)

In [ ]:
def baseline_index(i):
    if i in [2, 5]: return 0
    else: return 1

In [ ]:
def color_values(df, significant):
    better = "textcolor: {PineGreen} --rwrap;"
    worse = "textcolor: {red} --rwrap;"
    best = "textbf:--rwrap;"
    same = '' 
    df1 =  pd.DataFrame(same, index=df.index, columns=df.columns)
#     bb = df.apply(lambda x: [v == all_gs.iloc[i//7*7:i//7*7+7].to_numpy().max() for i, v in enumerate(x)], axis = 0)
    b = df.apply(lambda x: [i% 7 not in  [0, 1] and v > x.iloc[i//7*7 + baseline_index(i%7)] for i, v in enumerate(x)], axis = 0)
    w = df.apply(lambda x: [i% 7 not in  [0, 1] and v <  x.iloc[i//7*7 + baseline_index(i%7)] for i, v in enumerate(x)], axis = 0)
    return df1.mask(b, better).mask(w, worse).mask(~significant, same)

In [ ]:
def color(df):
    better = "textcolor: {PineGreen} --rwrap;"
    worse = "textcolor: {red} --rwrap;"
    best = "textbf:--rwrap;"
    same = '' 
    df1 =  pd.DataFrame(same, index=df.index, columns=df.columns)
#     bb = df.apply(lambda x: [v == all_gs.iloc[i//7*7:i//7*7+7].to_numpy().max() for i, v in enumerate(x)], axis = 0)
    b = df.apply(lambda x: [i% 7 not in  [0, 1] and v > x.iloc[i//7*7 + baseline_index(i%7)] for i, v in enumerate(x)], axis = 0)
    w = df.apply(lambda x: [i% 7 not in  [0, 1] and v <  x.iloc[i//7*7 + baseline_index(i%7)] for i, v in enumerate(x)], axis = 0)
#     return df1.mask(bb, best).mask(b, better).mask(w, worse)
    return df1.mask(b, better).mask(w, worse)

In [ ]:
from functools import partial

color = partial(color_values, significant=significant)

In [ ]:
l = l.apply(color, axis=None)

In [ ]:
l = l.highlight_max(axis=0, props="textbf:--rwrap;")

In [ ]:
print(l.to_latex(multirow_align="t"))

In [ ]:
all_gs.iloc[:,0:9].mean(1).sort_values()

## Figures

In [ ]:
dfs = [
    sst_df,
    qqp_df,
    squad_df,
    davidson_df,
    founta_df,
    sa_df,
    pi_df,
    rc_df,
    hsd_df,
]

In [ ]:
values = ["accuracy", "accuracy", "exact_match", "f1", "f1", "avg", "avg", "avg", "avg"]

In [ ]:
titles = ["SST2", "QQP", "SQuAD", "Davidson", "Founta", "SENT Suite", "PARA Suite", "READ Suite", "HATE suite"]

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(15,6), gridspec_kw=dict(wspace=0.1))
for row in range(2):
    for col in range(5):
        i = row*5 + col
        if row == 1 and col==4: 
            fig.delaxes(axes[row][col])
            break
        value = values[i]
        df = dfs[i].copy()
        if row == 0 and col != 2: df[value] = df[value] *100
        df = df.pivot(index=["model"], columns="method", values=value)
        df = df.sort_values(by="model", key=lambda x: x.map(order))
        df = df[sorted(df.columns, key= lambda item: order[item])]
        df.plot.bar(ax = axes[row, col], legend=0, ylim=(0,100), width=.8)
        axes[row,col].xaxis.label.set_visible(False)
        axes[row,col].title.set_text(titles[i])
        if row == 0: axes[row,col].tick_params('x', labelbottom=False)
        if col != 0: axes[row,col].tick_params('y', labelleft=False)
handles, labels = axes[1,3].get_legend_handles_labels()
fig.legend(handles, labels,  loc='upper left', bbox_to_anchor=(0.75, 0.45))

In [ ]:
fig.savefig(f"../specification-instruction-paper/media/dataset_results.pdf", bbox_inches = "tight")

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16,5), gridspec_kw=dict(wspace=0.1))
for row in range(2):
    for col in range(5):
        i = row*5 + col
        if row == 1 and col==4: 
            fig.delaxes(axes[row][col])
            break
        value = values[i]
        df = dfs[i].copy()
        if row == 0 and col != 2: df[value] = df[value] *100
        df = df.pivot(index=["method"], columns="model", values=value)
        df = df.sort_values(by="method", key=lambda x: x.map(order))
        df = df[sorted(df.columns, key= lambda item: order[item])]
        df.plot.bar(ax = axes[row, col], legend=0, ylim=(0,100), width=.8, rot=90)
        axes[row,col].xaxis.label.set_visible(False)
        axes[row,col].title.set_text(titles[i])
        if row == 0: axes[row,col].tick_params('x', labelbottom=False)
        if col != 0: axes[row,col].tick_params('y', labelleft=False)
handles, labels = axes[1,3].get_legend_handles_labels()
fig.legend(handles, labels,  loc='upper left', bbox_to_anchor=(0.75, 0.45))

In [ ]:
fig.savefig(f"../specification-instruction-paper/media/dataset_results_by_method.pdf", bbox_inches = "tight")

In [ ]:
suite_dfs = dfs[5:]

In [ ]:
suites_agg_df = pd.concat([x[["avg", "model", "method"]] if i == 0 else x["avg"] for i, x in enumerate(suite_dfs)], axis = 1)

In [ ]:
suites_agg_df.columns = ["avg_sa", "model", "method", "avg_pi", "avg_rc", "avg_hsd"]

In [ ]:
suites_agg_df["avg"] = suites_agg_df.mean(1)

In [ ]:
data_dfs = [dataset_results, suites_agg_df]

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(16,5), gridspec_kw=dict(wspace=0.1))
titles = ["Datasets", "Suites"]
for row in range(2):
    df = data_dfs[row].copy().reset_index()
    df = df.pivot(index=["model"], columns="method", values="avg")
    df = df.sort_values(by="model", key=lambda x: x.map(order))
    df = df[sorted(df.columns, key= lambda item: order[item])]
    df.plot.bar(ax = axes[row], legend=0, width=.8)
    axes[row].xaxis.label.set_visible(False)
    axes[row].title.set_text(titles[row])
    if row == 0: axes[row].tick_params('x', labelbottom=False)
#     if col != 0: axes[row,col].tick_params('y', labelleft=False)
handles, labels = axes[1].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, +1.05),
          fancybox=True, shadow=True, ncol=7)

In [ ]:
dataset_results = dataset_results.sort_values(by=["model", "method"], key=lambda x: x.map(order))

In [ ]:
suites_agg_df = suites_agg_df.sort_values(by=["model", "method"], key=lambda x: x.map(order))

In [ ]:
comps = []
for method in ["Task+Rules", "Task+Rules+Ex", "Task+Rules(chatGPT)+Ex", "Task+Rules+Rat", "Task+Rules+Ex+Rat"]:
    if "Ex" in method:
        base = "Task+Ex"
    else:
        base = "Task"
    comps.append(dataset_results.select_dtypes(include=np.number)[dataset_results.method == method] - dataset_results.select_dtypes(include=np.number)[dataset_results.method == base].values)
    comps[-1]["method"] = dataset_results[dataset_results.method == method].method
    comps[-1]["model"] = dataset_results[dataset_results.method == method].model
dataset_diffs = pd.concat(comps)

In [ ]:
comps = []
df = suites_agg_df.copy()
for method in ["Task+Rules", "Task+Rules+Ex", "Task+Rules(chatGPT)+Ex", "Task+Rules+Rat", "Task+Rules+Ex+Rat"]:
    if "Ex" in method:
        base = "Task+Ex"
    else:
        base = "Task"
    comps.append(df.select_dtypes(include=np.number)[df.method == method] - df.select_dtypes(include=np.number)[df.method == base].values)
    comps[-1]["method"] = df[df.method == method].method
    comps[-1]["model"] = df[df.method == method].model
suite_diffs = pd.concat(comps)

In [ ]:
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
comps[0]

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(15,6))
values = ['accuracy_sa', 'accuracy_pi', 'exact_match', 'f1_d', 'f1_f', 'avg_sa', 'avg_pi', 'avg_rc', 'avg_hsd']
titles = ["SST2", "QQP", "SQuAD", "Davidson", "Founta", "SENT Suite", "PARA Suite", "READ Suite", "HATE suite"]
dataset_diffs.columns = ['accuracy_sa', 'accuracy_pi', 'exact_match', 'f1_d', 'f1_f', "avg", "method", "model"]
data_dfs = [dataset_diffs, suite_diffs]
for row in range(2):
    for col in range(5):
        i = row*5 + col
        if row == 1 and col==4: 
            fig.delaxes(axes[row][col])
            break
        value = values[i]
        df = data_dfs[row].copy()
        df = df.pivot(index=["model"], columns="method", values=value)
        df = df.sort_values(by="model", key=lambda x: x.map(order))
        df = df[sorted(df.columns, key= lambda item: order[item])]
        df.plot.bar(ax = axes[row, col], legend=0, width=.8, color = cycle[2:])
        axes[row,col].xaxis.label.set_visible(False)
        axes[row,col].title.set_text(titles[i])
        axes[row, col].axhline(0, color='black', linewidth=.5)
#         axes[row, col].set_yscale("symlog")
        if row == 0: axes[row,col].tick_params('x', labelbottom=False)
#         if col != 0: axes[row,col].tick_params('y', labelleft=False)
handles, labels = axes[1,3].get_legend_handles_labels()
fig.legend(handles, labels,  loc='upper left', bbox_to_anchor=(0.75, 0.45))

In [ ]:
fig.savefig(f"../specification-instruction-paper/media/dataset_results_diffs.pdf", bbox_inches = "tight")

In [ ]:
data_dfs = [dataset_diffs[["model", "method", "avg"]], suite_diffs[["model", "method", "avg"]]]

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(12,6), gridspec_kw=dict(wspace=0.1))
titles = ["Datasets", "Suites"] 
for row in range(2):
    df = data_dfs[row].copy().reset_index()
    df = df.pivot(index=["model"], columns="method", values="avg")
    df = df.sort_values(by="model", key=lambda x: x.map(order))
    df = df[sorted(df.columns, key= lambda item: order[item])]
    df.plot.bar(ax = axes[row], legend=0, width=.8, color = cycle[2:])
    axes[row].xaxis.label.set_visible(False)
    axes[row].title.set_text(titles[row])
#     axes[row].set_yscale("symlog", linthresh = .01)
    axes[row].axhline(0, color='black', linewidth=.5)
    if row == 0: axes[row].tick_params('x', labelbottom=False)
#     if col != 0: axes[row,col].tick_params('y', labelleft=False)
handles, labels = axes[1].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, +1.05),
          fancybox=True, shadow=True, ncol=7)

In [ ]:
fig.savefig(f"../specification-instruction-paper/media/dataset_results_agg_diffs.pdf", bbox_inches = "tight")

### Rule generation quality vs functionality performances

In [ ]:
func_scores = pd.concat([sa_df, pi_df, rc_df, hsd_df], axis=1)

In [ ]:
rule_ratings = pd.read_csv("./data/chatGPTgeneratedRules/rule_ratings.csv").T

In [ ]:
tasks = 36*["SENT"] + 53*["PARA"] + 24*["READ"] + 29*["HATE"]

In [ ]:
rule_ratings.columns = rule_ratings.loc["Functionality"]

In [ ]:
rule_ratings

In [ ]:
func_scores.loc["ratings"] = rule_ratings.loc["Rating"]

In [ ]:
func_scores = func_scores.T

In [ ]:
func_scores = func_scores.iloc[:-3]

In [ ]:
func_scores = func_scores.drop(index=["avg", "model", "method", "score"])

In [ ]:
func_scores["task"] = tasks

In [ ]:
func_scores

In [ ]:
func_scores.groupby(["task", "ratings"]).mean()[["chatGPT_seen_example", "chatGPT_seen_from_chatGPT_example"]]

In [ ]:
func_scores.groupby(["task", "ratings"]).mean()[["chatGPT_seen_example", "chatGPT_seen_from_chatGPT_example"]]

In [ ]:
func_scores.groupby(["task", "ratings"]).mean()[["chatGPT_seen_example", "chatGPT_seen_from_chatGPT_example"]]

In [ ]:
func_scores[["chatGPT_seen_example", "chatGPT_seen_from_chatGPT_example"]]

In [ ]:
plt.rcParams.update({'font.size': 13})

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,5), gridspec_kw=dict(wspace=0.1), sharex=True, sharey=False)
axes = func_scores.boxplot(by="ratings", column=["chatGPT_baseline_example", "chatGPT_seen_from_chatGPT_example"], ax=axes)
axes[0].title.set_text("Task+Ex pass rates by rating")
axes[1].title.set_text("Task+Rules(cGPT)+Ex pass rates by rating")
axes[0].xaxis.label.set_visible(False)
axes[1].xaxis.label.set_text("Rating")
fig.suptitle("")

In [ ]:
fig.savefig(f"../specification-instruction-paper/media/funcVsGen.pdf", bbox_inches = "tight")

### Table functionality descriptions

In [ ]:
rule_ratings = rule_ratings.T

In [ ]:
rule_ratings

In [ ]:
sa_ratings = rule_ratings.iloc[:38].copy()

In [ ]:
pi_ratings = rule_ratings.iloc[38:38+53].copy()

In [ ]:
rc_ratings = rule_ratings.iloc[38+53:38+53+24].copy()

In [ ]:
hsd_ratings = rule_ratings.iloc[38+53+24:].copy()

In [ ]:
for task, ratings in zip(["sa", "pi", "rc", "hsd"], [sa_ratings, pi_ratings, rc_ratings, hsd_ratings]):
    with open(f"./data/{task}/suite/func_desc.pkl", "rb") as file:
        func_desc = pickle.load(file)
        order = {func.replace('"', ""): idx for idx, func in enumerate(ratings.Functionality.tolist())}
        od = collections.OrderedDict(sorted(func_desc.items(), key= lambda item: order[item[0].replace('"', "")]))
        ratings["h_descriptions"] = od.values()

In [ ]:
sa_ratings

In [ ]:
sa_table = sa_ratings[["Functionality", "h_descriptions", "Description", "Rating"]].style.format(escape="latex")

In [ ]:
print(sa_table.hide(axis=0).to_latex())

In [ ]:
pi_table = pi_ratings[["Functionality", "h_descriptions", "Description", "Rating"]].style.format(escape="latex")

In [ ]:
print(pi_table.hide(axis=0).to_latex())

In [ ]:
rc_table = rc_ratings[["Functionality", "h_descriptions", "Description", "Rating"]].style.format(escape="latex")

In [ ]:
print(rc_table.hide(axis=0).to_latex())

In [ ]:
hsd_ratings

In [ ]:
hsd_table = hsd_ratings[["Functionality", "h_descriptions", "Description", "Rating"]].style.format(escape="latex")

In [ ]:
print(hsd_table.hide(axis=0).to_latex())

In [ ]:
all_ratings = pd.concat([sa_ratings, pi_ratings, rc_ratings, hsd_ratings])

In [ ]:
all_ratings.Description.str.split().str.len().describe()

In [ ]:
all_ratings.h_descriptions.str.split().str.len().describe()

In [ ]:
all_ratings.iloc[90]["h_descriptions"]

In [ ]:
pi_ratings.iloc[0].Description

### Table rationale analysis

In [ ]:
rat_samples = pd.read_csv("./results/labelled_rat_samples.csv")

In [ ]:
rat_samples = rat_samples[["input", "pred", "Explanation correct", "Pred match expl.", "Why no correct", "Correct answer"]]

In [ ]:
rat_samples

In [ ]:
rat_samples["Explanation correct"].value_counts()

In [ ]:
21/40

In [ ]:
rat_samples["Pred match expl."].value_counts()

In [ ]:
rat_samples["Why no correct"].value_counts()

In [ ]:
rat_samples["Correct answer"].value_counts()

In [ ]:
rat_samples[rat_samples["Explanation correct"] == "1"]["Correct answer"].value_counts()

In [ ]:
rat_samples[rat_samples["Explanation correct"] == "0"]["Correct answer"].value_counts()

In [ ]:
rat_samples.groupby("Why no correct")["Correct answer"].value_counts()

## Qualitative analysis: samples that frequently change

In [ ]:
def process_df(df):
    df['model'] = [x.split("_")[0] for x in df.index]
    df['method'] = [x.split("_")[1] for x in df.index]
    df["score"] = df["method"]
    df['method'] = ["Task" if "baseline" in x else f"Task+Rules" for x in df.method]
    df['method'] = [y+"(chatGPT)"  if "from_chatGPT" in x else y for x,y in zip(df.index, df.method)]
    df['method'] =  [y+"+Ex"  if "example" in x else y for x,y in zip(df.index, df.method)]
    df['method'] =  [y+"+Rat"  if "rules" in x else y for x,y in zip(df.index, df.method)]
    df["model"] = df.model.str.split("-").str[-1]
    df = df.sort_values(["model", "method"], key=lambda x: x.map(order))
    return df

In [ ]:
def dataset_hits_df(task, result_path, dataset_test, metric, label_col="label"):
    results = load_results(result_path)
    results = {k: v for k, v in results.items() if "seen" in k or "baseline" in k}
    dataset_scores, preds = get_dataset_scores(task, results, dataset_test[label_col], metric)
    labels = dataset_test[label_col] if task != "rc" else [x["text"] for x in dataset_test["answers"]]
    hits = {}
    for model, pred in preds.items():
        if task != "rc":
            hits[model] = (np.array(pred) == np.array(labels)).astype(int)
        else:
            hits[model] = np.array([metric_max_over_ground_truths(exact_match_score, p["prediction_text"], label) for p, label in zip(pred, labels)]).astype(int)
    df = pd.DataFrame(hits).T
    df = process_df(df)
    return df, results

In [ ]:
def suite_hits_df(task, path, suite_test):
    hits = load_hits(path)
    all_hits = {}
    for model, hit in hits.items():
        all_hits.setdefault(model, []).extend([x for v in hit.values() for x in v])
    if task == "sa":
        suite_test = suite_test.filter(lambda x: x["functionality"] not in  ['"used to" should reduce', 'reducers'])
    test_id_to_row = collections.OrderedDict()
    if task != "hsd":
        for i, ex in enumerate(suite_test):
            test_id_to_row.setdefault((ex["functionality"], ex["test_id"]), []).append(i)
    df = pd.DataFrame(all_hits).T
    df = process_df(df)
    df = df[(df.score == "seen") | (df.score == "baseline")]
    return df, test_id_to_row

In [ ]:
def get_diffs(df):
    comps = [("Task+Rules", "Task"),
             ("Task+Rules+Ex", "Task+Ex"),
             ("Task+Rules(chatGPT)+Ex", "Task+Ex"),
             ("Task+Rules+Rat", "Task"),
             ("Task+Rules+Ex+Rat", "Task+Ex")]
    better = (df.select_dtypes(include=np.number)[(df.method == "Task+Rules") & (df.score=="seen")] > df.select_dtypes(include=np.number)[df.method == "Task"].values).sum()
    worse =  (df.select_dtypes(include=np.number)[(df.method == "Task+Rules") & (df.score=="seen")] < df.select_dtypes(include=np.number)[df.method == "Task"].values).sum()
    for a, b in comps[1:]:
        better += (df.select_dtypes(include=np.number)[(df.method == a) & (df.score=="seen")] > df.select_dtypes(include=np.number)[df.method == b].values).sum()
        worse += (df.select_dtypes(include=np.number)[(df.method == a) & (df.score=="seen")] < df.select_dtypes(include=np.number)[df.method == b].values).sum()
    return better, worse

### Sentiment analysis

In [ ]:
result_path = Path(f"./results/sa/sst2/")

In [ ]:
dataset_test = load_dataset("glue", "sst2")["validation"]

In [ ]:
metric = load_metric("glue","sst2")

In [ ]:
df, preds = dataset_hits_df("sa", result_path, dataset_test, metric)

In [ ]:
better, worse = get_diffs(df)

In [ ]:
better.value_counts(), worse.value_counts()

In [ ]:
better.argmax(), better.max()

In [ ]:
worse.argmax(), worse.max()

In [ ]:
dataset_test[377]

In [ ]:
worse[377]

In [ ]:
dataset_test[312]

In [ ]:
better[312]

In [ ]:
path = Path("results/sa/suite/")

In [ ]:
suite_test = get_suite(config.sa_path)["test"]

In [ ]:
df, test_id_to_row = suite_hits_df("sa", path, suite_test)

In [ ]:
better, worse = get_diffs(df)

In [ ]:
better.value_counts(), worse.value_counts()

In [ ]:
better.argmax(), better.max()

In [ ]:
worse.argmax(), worse.max()

In [ ]:
worse[worse == 15]

In [ ]:
suite_test[list(test_id_to_row.items())[4648][1]]

In [ ]:
worse[4648]

In [ ]:
suite_test[list(test_id_to_row.items())[7180][1]]

In [ ]:
better[7180]

In [ ]:
suite_test[list(test_id_to_row.items())[13924][1]]

In [ ]:
df[13924]

In [ ]:
better[13924]

### Paraphrase identification

In [ ]:
result_path = Path(f"./results/pi/qqp/")

In [ ]:
dataset_test = load_dataset("glue", "qqp")["validation"]

In [ ]:
metric = load_metric("glue","qqp")

In [ ]:
df, preds = dataset_hits_df("pi", result_path, dataset_test, metric)

In [ ]:
better, worse = get_diffs(df)

In [ ]:
better.value_counts(), worse.value_counts()

In [ ]:
better.argmax(), better.max()

In [ ]:
worse.argmax(), worse.max()

In [ ]:
dataset_test[22480]

In [ ]:
preds["chatGPT_seen_example_with_rules"][22480]

In [ ]:
worse[22480]

In [ ]:
dataset_test[19260]

In [ ]:
better[19260]

In [ ]:
preds["chatGPT_seen_example_with_rules"][19260]

In [ ]:
path = Path("results/pi/suite/")

In [ ]:
suite_test = get_suite(config.pi_path)["test"]

In [ ]:
df, test_id_to_row = suite_hits_df("pi", path, suite_test)

In [ ]:
better, worse = get_diffs(df)

In [ ]:
better.value_counts(), worse.value_counts()

In [ ]:
better.argmax(), better.max()

In [ ]:
worse.argmax(), worse.max()

In [ ]:
better[better==19]

In [ ]:
suite_test[list(test_id_to_row.items())[1327][1]]

In [ ]:
worse[1327]

In [ ]:
suite_test[list(test_id_to_row.items())[15009][1]]

In [ ]:
worse[15009]

In [ ]:
suite_test[list(test_id_to_row.items())[1621][1]]

In [ ]:
better[1621]

### Reading comprehension

In [ ]:
result_path = Path(f"./results/rc/squad/")

In [ ]:
sqadv2 = load_dataset("squad_v2")["validation"]

In [ ]:
dataset_test = load_dataset("squad")["validation"]

In [ ]:
metric = load_metric("squad")

In [ ]:
df, preds = dataset_hits_df("rc", result_path, dataset_test, metric, label_col="answers")

In [ ]:
better, worse = get_diffs(df)

In [ ]:
better.value_counts(), worse.value_counts()

In [ ]:
better.argmax(), better.max()

In [ ]:
worse.argmax(), worse.max()

In [ ]:
dataset_test[2054]

In [ ]:
worse[2054]

In [ ]:
preds["flan-t5-xxl_baseline_example"][2054]

In [ ]:
preds["chatGPT_seen_example_with_rules"][2054]

In [ ]:
dataset_test[7344]

In [ ]:
better[7344]

In [ ]:
preds["chatGPT_seen_example_with_rules"][7344]

In [ ]:
path = Path("results/rc/suite/")

In [ ]:
suite_test = get_suite(config.rc_path)["test"]

In [ ]:
df, test_id_to_row = suite_hits_df("rc", path, suite_test)

In [ ]:
better, worse = get_diffs(df)

In [ ]:
better.value_counts(), worse.value_counts()

In [ ]:
better.argmax(), better.max()

In [ ]:
worse.argmax(), worse.max()

In [ ]:
suite_test[list(test_id_to_row.items())[3108][1]]

In [ ]:
worse[3108]

In [ ]:
suite_test[list(test_id_to_row.items())[3145][1]]

In [ ]:
better[3145]

### Hate speech detection

In [ ]:
result_path = Path(f"./results/hsd/davidson2017/")

In [ ]:
dataset_test = load_hsd_dataset("davidson2017")["test"]

In [ ]:
metric = load_metric("glue", "qqp")

In [ ]:
df, preds = dataset_hits_df("hsd", result_path, dataset_test, metric)

In [ ]:
better, worse = get_diffs(df)

In [ ]:
better.value_counts(), worse.value_counts()

In [ ]:
better.argmax(), better.max()

In [ ]:
worse.argmax(), worse.max()

In [ ]:
better[better==10]

In [ ]:
dataset_test[89]

In [ ]:
preds["chatGPT_seen_example_with_rules"][89]

In [ ]:
worse[89]

In [ ]:
dataset_test[484]

In [ ]:
preds["chatGPT_seen_with_rules"][484]

In [ ]:
worse[484]

In [ ]:
dataset_test[1718]

In [ ]:
preds["chatGPT_seen_example_with_rules"][1718]

In [ ]:
better[1718]

In [ ]:
result_path = Path(f"./results/hsd/founta2018/")

In [ ]:
dataset_test = load_hsd_dataset("founta2018")["test"]

In [ ]:
df, preds = dataset_hits_df("hsd", result_path, dataset_test, metric)

In [ ]:
better, worse = get_diffs(df)

In [ ]:
better.value_counts(), worse.value_counts()

In [ ]:
better.argmax(), better.max()

In [ ]:
worse.argmax(), worse.max()

In [ ]:
dataset_test[9520]

In [ ]:
preds["chatGPT_seen_example_with_rules"][9520]

In [ ]:
preds["chatGPT_seen_with_rules"][9520]

In [ ]:
worse[9520]

In [ ]:
preds["chatGPT_seen_example_with_rules"][1021]

In [ ]:
worse[worse==12]

In [ ]:
dataset_test[1021]

In [ ]:
better[1021]

In [ ]:
dataset_test[4717]

In [ ]:
better[4717]

In [ ]:
dataset_test[8296]

In [ ]:
better[8296]

In [ ]:
dataset_test[8492]

In [ ]:
better[8492]

In [ ]:
preds["chatGPT_seen_example_with_rules"][4717]

In [ ]:
preds["chatGPT_seen_with_rules"][4717]

In [ ]:
path = Path("results/hsd/suite/")

In [ ]:
suite_test = get_suite(config.hatecheck_path, hateCheck=True)["test"]

In [ ]:
df, preds = dataset_hits_df("hsd", path, suite_test, metric, label_col="label_gold")

In [ ]:
better, worse = get_diffs(df)

In [ ]:
better.value_counts(), worse.value_counts()

In [ ]:
better.argmax(), better.max()

In [ ]:
worse.argmax(), worse.max()

In [ ]:
better[better==13]

In [ ]:
suite_test[762]

In [ ]:
worse[762]

In [ ]:
preds["chatGPT_seen_example_with_rules"][762]

In [ ]:
suite_test[846]

In [ ]:
worse[846]

In [ ]:
suite_test[859]

In [ ]:
worse[859]

In [ ]:
suite_test[872]

In [ ]:
better[872]